In [1]:
# NS
#optiver-ver (for ensemble)

#optiver-ver-nn-modeling/inference-v1
#optiver-ver-rnn-modeling/inference-v1
#optiver-ver-lgb-modeling/inference-v1


import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
# import lightgbm as lgb
# import xgboost as xgb
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

from scipy.stats import hmean

from sklearn.ensemble import HistGradientBoostingRegressor
import itertools
import pickle
import joblib
from itertools import combinations
from tqdm import tqdm
import gc
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, GaussianNoise
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import concatenate,Dropout
import pickle
from tensorflow.keras.models import load_model
import os
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.callbacks import Callback
import random
from tensorflow.keras.layers import Input, Embedding, Lambda, Reshape, LSTM, Dense, BatchNormalization, Dropout, concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.layers import ZeroPadding1D
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import RepeatVector

from tensorflow.keras.layers import Input, Embedding, Lambda, Reshape, LSTM, Dense, BatchNormalization, Dropout, concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.layers import ZeroPadding1D, Activation
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import MaxPooling1D, AveragePooling1D
from tensorflow.keras.layers import Add
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from scipy.stats import hmean

from sklearn.ensemble import HistGradientBoostingRegressor
import itertools
import pickle
import joblib
from itertools import combinations
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, GaussianNoise
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import concatenate,Dropout
import pickle
from tensorflow.keras.models import load_model
import os 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.callbacks import Callback
import random
#---------------------------------------------------------------------- setup dashboard ------------------------------------------------------------

jointquant          = True
is_inference        = False
load_models         = False
run_pipeline        = True
train_models        = True
is_rnn              = True   
online_learning     = True




#public-validation
dates_train = ["2018-01-01","2021-06-30"]
dates_test = ["2021-06-30","2021-06-30"]

#full-inference
#dates_train = [0,480]
#dates_test = [-1,-1]


num_models ={'rnn':1}


if jointquant:

    train_path = 'A:\joinquant/train.csv'
    models_path = "infermodel2/"



if dates_train[1]!="2023-06-30":
    models_path = "vals-630-2/"
#---------------------------------------------------------------------- setup dashboard ------------------------------------------------------------

simulation_path = "vals-258/"

In [2]:
rnn_ep         =45
rnn_lr         = 0.001
rnn_bs         = 2**12
window_size    = 2

In [3]:
def read_and_concatenate(year):
    # 文件路径
    path_1000 = f"factor/1000/{year}_1000.csv.gz"
    path_1330 = f"factor/1330/{year}_1330.csv.gz"
    
    # 读取数据
    df_1000 = pd.read_csv(path_1000, compression='gzip')
    df_1330 = pd.read_csv(path_1330, compression='gzip')

    # 为每个DataFrame添加时间列
    df_1000['time'] = '10:00:00'
    df_1330['time'] = '13:30:00'
    
    # 使用concat而不是merge
    concatenated_df = pd.concat([df_1000, df_1330], axis=0)
    
    # 可以选择排序，如果需要按照日期和代码顺序排列
    concatenated_df.sort_values(by=['date', 'code', 'time'], inplace=True)

    return concatenated_df

# 处理每一年的数据
years = [2018,2019,2020,2021]
all_data = pd.concat([read_and_concatenate(year) for year in years], axis=0)

# all_data 现在包含了所有年份的数据，每个时间点的数据并排放置

path_lable = "label.csv.gz"
lable1 = pd.read_csv(path_lable, compression = "gzip") 
# 合并 all_data 和 label DataFrame
train = pd.merge(all_data, lable1, on=['date', 'code',"time"], how='inner')

# merged_df 现在包含了合并后的数据
del lable1 , all_data
gc.collect()

0

In [4]:
train = train.drop('ret_next_5_close_alpha', axis=1)

In [5]:
def convert_price_cols_float32(df):

    # all Columns 
    all_columns = [col for col in df.columns if 'factor' in col]
    df[all_columns] = df[all_columns].astype('float32')

    return df

#train.to_csv('A:\joinquant/train.csv', index=False)

In [6]:
import pandas as pd



# 按照股票ID和日期进行分组，并计算每组中的唯一时间点数量
grouped = train.groupby(['date', 'code'])['time'].nunique()

# 找出只有一个时间点的组（即缺失另一个时间点的情况）
missing_one_period = grouped[grouped == 1]

# 获取缺失一个时间点的组合的索引
missing_index = missing_one_period.index

# 使用索引剔除这些组
filtered_train = train.set_index(['date', 'code'])
filtered_train = filtered_train.drop(index=missing_index)

# 重置索引
filtered_train = filtered_train.reset_index()

# 计算剔除缺失数据后的唯一天数
unique_days_after_filtering = filtered_train['date'].nunique()
print(f"剔除缺失数据后的唯一天数: {unique_days_after_filtering}")
#filtered_train.to_csv('A:\joinquant/train.csv', index=False)

剔除缺失数据后的唯一天数: 973


In [7]:

train = filtered_train
del filtered_train
gc.collect()

0

In [8]:
#train = pd.read_csv(train_path)

print(f"converting column names")
train = train.rename(columns = {'ret_next_close_alpha':'target','code':'stock_id','date':'date_id'})

nan_count = train['target'].isna().sum()
print(f"The 'target' column has {nan_count} NaN values.")
target_median = train['target'].median()
train['target'].fillna(target_median, inplace=True)

print(f"converting prices columns to float32 values.")
train = convert_price_cols_float32(train)

print(f"transforming datetime into float")
time_replace_dict = {
    '13:30:00': 13.5,
    '10:00:00': 10
    }
train['time'] = train['time'].replace(time_replace_dict)
print("done")
# ----------------------------- Reading train data -------------------------

converting column names
The 'target' column has 2 NaN values.
converting prices columns to float32 values.
transforming datetime into float
done


In [9]:
train.head()

,date_id,stock_id,factor000,factor001,factor002,factor003,factor004,factor005,factor006,factor007,...,factor492,factor493,factor494,factor495,factor496,factor497,factor498,factor499,time,target
0,2018-01-02,000001.XSHE,-0.8850,1.4940,1.6930,1.7250,-0.3691,-1.2340,-0.48300,1.670,...,-1.164,0.0878,0.4082,-0.3967,1.5960,1.3560,-1.479,-1.6140,10.0,-0.056046
1,2018-01-02,000001.XSHE,-0.1461,-0.1818,-0.1921,0.9470,-1.5080,-0.3245,-1.46300,1.485,...,-1.632,0.5650,0.8916,0.8237,1.5450,1.4120,-1.530,0.6973,13.5,-0.044346
2,2018-01-02,000002.XSHE,1.1860,1.5450,1.3880,1.6310,-1.2130,-1.4330,-1.54900,1.675,...,-1.066,1.0670,-0.4070,-0.6553,1.3470,0.9307,-1.685,-1.5620,10.0,-0.028517
3,2018-01-02,000002.XSHE,-1.2690,1.6070,0.8135,1.5840,-1.2070,1.4150,-1.58400,1.609,...,-1.668,0.9995,1.3390,-1.0490,1.5720,1.0160,-1.607,0.3245,13.5,-0.024816
4,2018-01-02,000004.XSHE,-1.1990,0.2200,0.1958,-0.3176,0.7140,0.8413,-0.08673,-1.441,...,-1.259,-0.2119,-0.5186,0.2637,-1.4375,-0.9595,0.516,0.6377,10.0,0.055232


In [10]:
#@title functions

def split_by_date(df, dates):

    df_start, df_end = dates
    df = df[(df['date_id'] >= df_start) & (df['date_id'] <=df_end)].reset_index(drop=True)

    return df



def lag_function(df, columns_to_lag, numbers_of_days_to_lag):

    df_indexed = df.set_index(['stock_id', 'time', 'date_id'])

    for column_to_lag in columns_to_lag:
        for number_days_to_lag in numbers_of_days_to_lag:
            df_indexed[f'lag{number_days_to_lag}_{column_to_lag}'] = df_indexed.groupby(level=['stock_id', 'time'])[column_to_lag].shift(number_days_to_lag)

    df_indexed.reset_index(inplace=True)

    return df_indexed




def compute_imbalances(df_, columns, prefix = ''):
    """Computes the differences and imbalances for pairs of columns and stores them in the DataFrame."""
    df = df_.copy()
    for col1, col2 in combinations(columns, 2):

        # Sort the columns lexicographically to ensure consistent ordering
        col1, col2 = sorted([col1, col2])

        # Compute imbalance directly without creating a temporary difference column
        total = df[col1] + df[col2]
        imbalance_column_name = f'{col1}_{col2}_imb{prefix}'

        # Ensure we don't divide by zero
        df[imbalance_column_name] = (df[col1] - df[col2]).divide(total, fill_value=np.nan)

    return df



def save_pickle(data, file_path):

    # Create the directory if it doesn't exist
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Save the pickle file
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)

    print(f"Data saved to {file_path}")
    #example: save_pickle(all_data, 'k8/all_data.pkl')

def load_pickle(file_path):

    # Load and return the data from the pickle file
    if os.path.exists(file_path):
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
        return data
    else:
        raise FileNotFoundError(f"No such file: {file_path}")

    #example: loaded_data = load_pickle('k8/all_data.pkl')

In [11]:
num_of_target_lags  = 3
target_lags         = list(range(1,num_of_target_lags+1))


def feature_pipeline(df):

    if df.empty:
        return pd.DataFrame()


    # df = compute_imbalances(df, columns_sizes,prefix='_sz_')
    # df = compute_imbalances(df, columns_prices,prefix = '_pr_')


    print(f"lagging target column for {len(target_lags)} lags.")
    df = lag_function(df, ['target'], target_lags)



    print("Done...")

    return df

In [12]:
def make_predictions(models, X_test,model = 'nn'):
    if model == 'nn':
        all_predictions = [model.predict(X_test, batch_size=16384) for model in models]
    if model == 'lgb' or model == 'xgb' or model == 'cat':
        all_predictions = [model.predict(X_test) for model in models]
    prediction = np.mean(all_predictions, axis=0)
    return prediction

def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

class BestScoresCallback(Callback):
    def __init__(self):
        super().__init__()
        self.best_train_loss = float('inf')
        self.best_val_loss = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        train_loss = logs.get('loss', float('inf'))
        val_loss = logs.get('val_loss', float('inf'))

        if train_loss < self.best_train_loss:
            self.best_train_loss = train_loss
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss

    def on_train_end(self, logs=None):
        print(f"Best training loss: {self.best_train_loss}, Best validation loss: {self.best_val_loss}")

In [13]:
#@title RNN second pass

def precompute_sequences(stock_data, window_size, rnn_numerical_features, rnn_categorical_features):
    # Convert DataFrame columns to NumPy arrays
    stock_data_num = stock_data[rnn_numerical_features].values
    stock_data_cat = stock_data[rnn_categorical_features].values

    # Pre-compute all sequences
    all_sequences_num = [stock_data_num[max(0, i - window_size + 1):i + 1] for i in range(len(stock_data))]
    all_sequences_cat = [stock_data_cat[max(0, i - window_size + 1):i + 1] for i in range(len(stock_data))]

    # Add padding if necessary
    padded_sequences_num = [np.pad(seq, ((window_size - len(seq), 0), (0, 0)), 'constant') for seq in all_sequences_num]
    padded_sequences_cat = [np.pad(seq, ((window_size - len(seq), 0), (0, 0)), 'constant') for seq in all_sequences_cat]

    # Combine numerical and categorical features
    combined_sequences = np.array([np.concatenate([num, cat], axis=-1)
                                   for num, cat in zip(padded_sequences_num, padded_sequences_cat)])

    # Extract targets
    targets = stock_data['target'].values

    return combined_sequences, targets

def get_sequence(precomputed_data, time_step):
    combined_sequences, targets = precomputed_data
    return combined_sequences[time_step], targets[time_step]



def create_batches(data, window_size, rnn_numerical_features, rnn_categorical_features, max_time_steps= 2):

    grouped = data.groupby(['stock_id', 'date_id'])
    all_batches = []
    all_targets = []

    for _, group in tqdm(grouped, desc="Processing groups"):
        # Precompute sequences for the current group
        precomputed_data = precompute_sequences(group, window_size, rnn_numerical_features, rnn_categorical_features)

        # Initialize containers for group sequences and targets
        group_sequences = []
        group_targets = []

        # Iterate over the time steps and retrieve precomputed sequences
        for time_step in range(max_time_steps):
            sequence, target = get_sequence(precomputed_data, time_step)
            if sequence.size > 0:
                group_sequences.append(sequence)
                group_targets.append(target)

        # Extend the main batches with the group's sequences and targets
        all_batches.extend(group_sequences)
        all_targets.extend(group_targets)

    return all_batches, all_targets




def compute_last_sequence(group, window_size, rnn_numerical_features, rnn_categorical_features):
    # Convert DataFrame columns to NumPy arrays
    stock_data_num      = group[rnn_numerical_features].values
    stock_data_cat      = group[rnn_categorical_features].values
    stock_data_target   = group['target'].values

    # Find the index of the target second
    target_index = len(group) - 1

    # Extract the sequence for the target index
    sequence_num = stock_data_num[max(0, target_index - window_size + 1):target_index + 1]
    sequence_cat = stock_data_cat[max(0, target_index - window_size + 1):target_index + 1]

    # Add padding if necessary
    padded_sequence_num = np.pad(sequence_num, ((window_size - len(sequence_num), 0), (0, 0)), 'constant')
    padded_sequence_cat = np.pad(sequence_cat, ((window_size - len(sequence_cat), 0), (0, 0)), 'constant')

    # Combine numerical and categorical features
    combined_sequence = np.concatenate([padded_sequence_num, padded_sequence_cat], axis=-1)

    # Extract target
    target = stock_data_target[-1]

    return combined_sequence, target


def create_last_batches(data, window_size, rnn_numerical_features, rnn_categorical_features):

    grouped = data.groupby(['stock_id'])
    all_batches = []
    all_targets = []

    for _, group in grouped:
        # Compute the sequence for the last data point in the current group
        last_sequence, last_target = compute_last_sequence(group, window_size, rnn_numerical_features, rnn_categorical_features)

        # Check if the sequence is valid (i.e., not empty)
        if last_sequence.size > 0:
            all_batches.append(last_sequence)
            all_targets.append(last_target)

    return all_batches, all_targets





def second_pass_for_rnn(df, rnn_numerical_features, rnn_categorical_features, window_size, is_inference=False):
    # Check if the DataFrame is empty
    global rnn_scaler,rnn_medians

    if df.empty:
        return None, None

    # Work on a copy of the DataFrame to avoid changing the original df
    df_copy = df.copy()

    # Standard scaling for numerical features
    if is_inference:
        df_copy.fillna(rnn_medians, inplace=True)
        df_copy[rnn_numerical_features] = rnn_scaler.transform(df_copy[rnn_numerical_features])



    if is_inference:
        df_copy_batches, df_copy_targets = create_last_batches(df_copy, window_size, rnn_numerical_features, rnn_categorical_features)
    else:
        df_copy_batches, df_copy_targets = create_batches(df_copy, window_size, rnn_numerical_features, rnn_categorical_features)

    df_copy_batches = np.array(df_copy_batches)
    df_copy_targets = np.array(df_copy_targets)


    return df_copy_batches, df_copy_targets


def online_pass_for_rnn(df, rnn_numerical_features, rnn_categorical_features, window_size):
    # Check if the DataFrame is empty
    global rnn_scaler,rnn_medians

    if df.empty:
        return None, None

    # Work on a copy of the DataFrame to avoid changing the original df
    df_copy = df.copy()

    # Standard scaling for numerical features
    df_copy.fillna(rnn_medians, inplace=True)
    df_copy[rnn_numerical_features] = rnn_scaler.transform(df_copy[rnn_numerical_features])


    grouped = df_copy.groupby(['stock_id'])
    all_batches = []
    all_targets = []

    for _, group in tqdm(grouped, desc="Processing groups"):
        # Precompute sequences for the current group
        precomputed_data = precompute_sequences(group, window_size, rnn_numerical_features, rnn_categorical_features)

        # Initialize containers for group sequences and targets
        group_sequences = []
        group_targets = []

        # Iterate over the time steps and retrieve precomputed sequences
        for time_step in range(2):
            sequence, target = get_sequence(precomputed_data, time_step)
            if sequence.size > 0:
                group_sequences.append(sequence)
                group_targets.append(target)

        # Extend the main batches with the group's sequences and targets
        all_batches.extend(group_sequences)
        all_targets.extend(group_targets)

    df_batches = np.array(all_batches)
    df_targets = np.array(all_targets)


    return df_batches, df_targets

In [14]:
#@title RNN model
from tensorflow.keras.layers import Input, Embedding, Lambda, Reshape, LSTM, Dense, BatchNormalization, Dropout, concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.layers import ZeroPadding1D
def apply_conv_layers(input_layer, kernel_sizes, filters= 16, do_ratio=0.5):
    conv_outputs = []

    for kernel_size in kernel_sizes:
        conv_layer = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same')(input_layer)
        conv_layer = BatchNormalization()(conv_layer)
        conv_layer = Dropout(do_ratio)(conv_layer)

        shortcut = conv_layer

        conv_layer = Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(conv_layer)
        conv_layer = BatchNormalization()(conv_layer)
        conv_layer = Activation('relu')(conv_layer)

        # Add the output of the first Conv1D layer
        conv_layer = Add()([conv_layer, shortcut])
        conv_outputs.append(conv_layer)


    concatenated_conv = Concatenate(axis=-1)(conv_outputs)
    flattened_conv_output = Flatten()(concatenated_conv)

    return flattened_conv_output

def create_rnn_model_with_residual(window_size, numerical_features, initial_learning_rate=0.001):
    
    categorical_features = 'time'
    categorical_uniques  = {'time':2}
    embedding_dim        = {'time':2}

    input_layer = Input(shape=(window_size, len(numerical_features) + 1), name="combined_input")

    # Split the input into numerical and categorical parts
    numerical_input = Lambda(lambda x: x[:, :, :-1], name="numerical_part")(input_layer)
    categorical_input = Lambda(lambda x: x[:, :, -1:], name="categorical_part")(input_layer)

    # Function to create a difference layer for a given lag
    def create_difference_layer(lag):
        return Lambda(lambda x: x[:, lag:, :] - x[:, :-lag, :], name=f"difference_layer_lag{lag}")

    # List to store all difference layers
    difference_layers = []

    # Create difference layers for each lag
    for lag in range(1, window_size):
        diff_layer = create_difference_layer(lag)(numerical_input)
        padding = ZeroPadding1D(padding=(lag, 0))(diff_layer)  # Add padding to the beginning of the sequence
        difference_layers.append(padding)

    

    combined_diff_layer = Concatenate(name="combined_difference_layer")(difference_layers) 
    
    enhanced_numerical_input = Concatenate(name="enhanced_numerical_input")([numerical_input, combined_diff_layer])

#     concat_input = Concatenate(name="concatenated_input")([enhanced_numerical_input, categorical_input])

    # Embedding for categorical part
    vocab_size, embedding_dim = categorical_uniques[categorical_features], embedding_dim[categorical_features]
    embedding = Embedding(vocab_size, embedding_dim, input_length=window_size)(categorical_input)
    embedding = Reshape((window_size, -1))(embedding)


  
    # Concatenate numerical input and embedding
    lstm_input = concatenate([enhanced_numerical_input, embedding], axis=-1)

    # Initialize a list to hold the outputs of each LSTM layer
#     lstm_outputs = []

    # First LSTM layer
    lstml = LSTM(64, return_sequences=False)(lstm_input)
    lstml = BatchNormalization()(lstml)
    lstml = Dropout(0.3)(lstml)
    
    dense_output = lstml
    dense_sizes = [512, 256, 128, 64, 32]
    do_ratio = 0.3
    for size in dense_sizes:
        dense_output = Dense(size, activation='swish')(dense_output)
        dense_output = BatchNormalization()(dense_output)
        dense_output = Dropout(do_ratio)(dense_output)

    # Output layer
    output = Dense(1, name='output_layer')(dense_output)

    # Learning rate schedule
    lr_schedule = ExponentialDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=10000,
        decay_rate=0.7,
        staircase=True)

    # Create and compile the model
    model = Model(inputs=input_layer, outputs=output)
    optimizer = Adam(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer, loss="mean_absolute_error")

    return model

In [15]:
#@title runnig pipeline

excluded_columns = ['date_id','target']

if run_pipeline:

    train_eng = feature_pipeline(train)
    del train
    gc.collect()
    if is_rnn:
        excluded_columns = excluded_columns + ['stock_id']

        features = [col for col in train_eng.columns if col not in excluded_columns]
        rnn_categorical_features =  ['time']
        rnn_numerical_features = [feat for feat in features if feat not in rnn_categorical_features]
        print("we have {} numerical and {} categorical".format(len(rnn_numerical_features),len(rnn_categorical_features)))


        rnn_scaler = StandardScaler()
        rnn_medians = train_eng.median(numeric_only = True)

        train_eng.fillna(rnn_medians, inplace=True)
        train_eng[rnn_numerical_features] = rnn_scaler.fit_transform(train_eng[rnn_numerical_features])


        rnn_all_data = {
            "rnn_scaler": rnn_scaler,
            "rnn_medians": rnn_medians,
            "rnn_categorical_features": rnn_categorical_features,
            "rnn_numerical_features": rnn_numerical_features
        }

        save_pickle(rnn_all_data, f'{models_path}rnn_all_data.pkl')
        print("Pipline Done!")

    print("splitting train data")
    train_data       = split_by_date(train_eng, dates_train)
    print("splitting test data")
    test_data        = split_by_date(train_eng, dates_test)
    print("number of dates in train = {} , number of dates in test {}".format (train_data['date_id'].nunique(),test_data['date_id'].nunique()))

    cleaning = True
    if cleaning:
        import gc
        #del train
        del train_eng
        gc.collect()

lagging target column for 3 lags.
Done...
we have 503 numerical and 1 categorical
Data saved to vals-630-2/rnn_all_data.pkl
Pipline Done!
splitting train data
splitting test data
number of dates in train = 848 , number of dates in test 1


In [16]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

import tensorflow as tf
import tensorflow.keras.backend as K
print('Using TensorFlow version',tf.__version__)


Using TensorFlow version 2.9.0


In [17]:
#@title prepare train models
if train_models:

    if test_data.empty:
        print("--------------test data is empty so adjusting the last date for test-----------------")
        test_data = train_data.query("date_id == 2021-12-31").copy()

    if is_rnn:

        train_batches, train_targets = second_pass_for_rnn(train_data, rnn_numerical_features, rnn_categorical_features, window_size)
        del train_data
        gc.collect()
        test_batches, test_targets  = second_pass_for_rnn(test_data, rnn_numerical_features, rnn_categorical_features, window_size)
        del test_data
        print(f"train batches shape:{train_batches.shape}")

Processing groups: 100%|██████████| 3711/3711 [00:04<00:00, 896.46it/s]


train batches shape:(5626764, 2, 504)


In [18]:
gc.collect()

20371

In [19]:
#@title training models
# train_data = flatten_outliers(train_data, 'target', lower_quantile=0.01, upper_quantile=0.99)
if train_models:

        directory = os.path.dirname(models_path)
        if not os.path.exists(directory):
            os.makedirs(directory)


        if is_rnn:

            callbacks = [BestScoresCallback()]  # Always include BestScoresCallback
            if dates_train[1] != "2021-06-30":
                early_stopping = EarlyStopping(monitor='val_loss', patience=70, restore_best_weights=False)
                callbacks.append(early_stopping)

            if True or tpu_strategy:
                # with tpu_strategy.scope():
                    rnn_models = []
                    for i in range(num_models['rnn']):
                        #resetgpu()
                        print(f"Training rnn model {i+1} out of {num_models['rnn']} with seed {42+i}")
                        print("---------------------------------------")
                        set_all_seeds(43+i)

                        rnn_model = create_rnn_model_with_residual(window_size, rnn_numerical_features, initial_learning_rate=rnn_lr)
                        history = rnn_model.fit(train_batches, train_targets, validation_data=(test_batches, test_targets), epochs=rnn_ep, batch_size=rnn_bs, callbacks=callbacks)
                        print("---------------------------------------")
                        rnn_model.save(f'{models_path}rnn_model_seed_{i+1}.h5')
                        rnn_models.append(rnn_model)
                     
                    predictions =  make_predictions(rnn_models, test_batches,model = 'nn')
                    print(f"Ensemble Mean Absolute Error: {mean_absolute_error(test_targets, predictions):.4f}")

Training rnn model 1 out of 1 with seed 42
---------------------------------------


2024-01-22 00:20:44.634126: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-22 00:20:45.421925: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46714 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:81:00.0, compute capability: 8.6
2024-01-22 00:20:45.422713: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46714 MB memory:  -> device: 1, name: NVIDIA A40, pci bus id: 0000:c1:00.0, compute capability: 8.6
2024-01-22 00:20:54.532603: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 22687112448 exceeds 10% of free system memory.


Epoch 1/45


2024-01-22 00:21:05.091961: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


   6/1374 [..............................] - ETA: 16s - loss: 1.0675    

2024-01-22 00:21:05.702576: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1374/1374 [==============================] - 21s 13ms/step - loss: 0.0561 - val_loss: 0.0241
Epoch 2/45
1374/1374 [==============================] - 17s 12ms/step - loss: 0.0193 - val_loss: 0.0227
Epoch 3/45
1374/1374 [==============================] - 17s 12ms/step - loss: 0.0180 - val_loss: 0.0230
Epoch 4/45
1374/1374 [==============================] - 17s 13ms/step - loss: 0.0178 - val_loss: 0.0222
Epoch 5/45
1374/1374 [==============================] - 19s 14ms/step - loss: 0.0172 - val_loss: 0.0217
Epoch 6/45
1374/1374 [==============================] - 18s 13ms/step - loss: 0.0170 - val_loss: 0.0217
Epoch 7/45
1374/1374 [==============================] - 16s 12ms/step - loss: 0.0169 - val_loss: 0.0216
Epoch 8/45
1374/1374 [==============================] - 17s 12ms/step - loss: 0.0168 - val_loss: 0.0215
Epoch 9/45
1374/1374 [==============================] - 17s 12ms/step - loss: 0.0167 - val_loss: 0.0217
Epoch 10/45
1374/1374 [==============================] - 17s 13ms/step - lo